## Review WEIS CSV Files

In [1]:
# Load all the tools
import numpy as np

from openfast_io import FileTools

import pandas as pd


import matplotlib.pyplot as plt

import openmdao.api as om

import matplotlib
# font = {
# #         'family' : 'normal',
# #         'weight' : 'bold',
#         'size'   : 14
#        }

# matplotlib.rc('font', **font)



In [2]:
# Output from examples/06_IEA-15-240-RWT/weis_driver_umaine_semi.py
turb = pd.read_csv('outputs/IEA-22-280-RWT.csv')

In [3]:
turb

,variables,type,units,values,description
0,aeroelastic.V_cutin,input,m/s,3.0,Minimum wind speed where turbine operates (cut...
1,aeroelastic.V_cutout,input,m/s,25.0,Maximum wind speed where turbine operates (cut...
2,aeroelastic.Vrated,input,m/s,11.017760490140262,rated wind speed
3,aeroelastic.hub_height,input,m,170.0,hub height
4,aeroelastic.Rtip,input,m,142.33649355558555,dimensional radius of tip
...,...,...,...,...,...
5996,xf.cd_interp_flaps,output,NaN,"[[[[0.35], [0.35], [0.35], [0.35], [0.35], [0....",4D array with the drag coefficients of the air...
5997,xf.cm_interp_flaps,output,NaN,"[[[[-0.00010000000000000005], [-0.000100000000...",4D array with the moment coefficients of the a...
5998,xf.flap_angles,output,deg,"[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...",3D array with the flap angles of the airfoils....
5999,xf.Re_loc,output,NaN,"[[[1216207.9390627919], [1216207.9390627919], ...",3D array with the Re. Dimension 0 is along the...


In [4]:
# Compare Outputs

In [5]:
comp_files = [
    'outputs/IEA-22-280-RWT.csv', 
    # 'outputs/refturb_output_04_NREL5MW_OC4_semi.csv', # generated from examples/04_NREL5MW_OC4_semi/weis_driver.py
    
]

# only works on single values for now
comp_values = [
    'floatingse.platform_mass',
    'financese.bos_per_kW',
    'financese_post.plant_aep',
    'financese_post.lcoe',
]

comp = {}
for value in comp_values:
    comp[value] = []

turbs = []

for file in comp_files:
    turb = pd.read_csv(file)
    turbs.append(turb)
    
    
    
    for value in comp_values:
        i_val = turb.variables == value
        a = turb[i_val]['values'].to_numpy()
        val = float(a[0][1:-1])
        comp[value].append(val)
    
    

In [6]:
comp

{'floatingse.platform_mass': [1190161.93890953],
 'financese.bos_per_kW': [412.80027878633],
 'financese_post.plant_aep': [552937295.410978],
 'financese_post.lcoe': [0.0788252768887259]}

In [7]:
# Some plots I made a while back

# x = np.arange(2)

# lcoe_perc = 100* (np.array(comp['financese_post.lcoe'])/comp['financese_post.lcoe'][0] - 1)
# ptfm_perc = 100* (np.array(comp['floatingse.platform_mass'])/comp['floatingse.platform_mass'][0] - 1)
# bos_perc = 100* (np.array(comp['financese.bos_per_kW'])/comp['financese.bos_per_kW'][0] - 1)
# aep_perc = 100* (np.array(comp['financese_post.plant_aep'])/comp['financese_post.plant_aep'][0] - 1)


# lcoe =np.array(comp['financese_post.lcoe'])
# ptfm =np.array(comp['floatingse.platform_mass'])
# bos =np.array(comp['financese.bos_per_kW'])
# aep =np.array(comp['financese_post.plant_aep'])

# comp['financese_post.lcoe'][0]


# fig, axs = plt.subplots(2,2)
# # [a.grid() for a in axs.flatten()]

# buf = 0.05
# axs[0,0].set_ylabel('Platform Mass (kg)')
# axs[0,0].bar(x,ptfm)
# axs[0,0].set_ylim([ptfm.min()*(1-buf),ptfm.max()*(1+buf)])

# for i,p in enumerate(axs[0,0].patches[1:]):
#     if p.get_height() > 0:
#         va = 'bottom'
#     else:
#         va = 'top'
#     axs[0,0].annotate(f'{ptfm_perc[i+1]:0.2f}%', (p.get_x()+p.get_width()/2, p.get_height()), fontsize=14, ha='center', va=va)



# axs[0,1].set_ylabel('BOS ($/kW)')
# axs[0,1].bar(x,bos)
# axs[0,1].set_ylim([bos.min()*(1-buf),bos.max()*(1+buf)])

# for i,p in enumerate(axs[0,1].patches[1:]):
#     if p.get_height() > 0:
#         va = 'bottom'
#     else:
#         va = 'top'
#     axs[0,1].annotate(f'{bos_perc[i+1]:0.2f}%', (p.get_x()+p.get_width()/2, p.get_height()), fontsize=14, ha='center', va=va)

    

# buf = 0.005
# axs[1,0].set_ylabel('AEP (kWh)')
# axs[1,0].bar(x,aep)
# axs[1,0].set_ylim([aep.min()*(1-buf),aep.max()*(1+buf)])

# for i,p in enumerate(axs[1,0].patches[1:]):
#     if p.get_height() > 0:
#         va = 'bottom'
#     else:
#         va = 'top'
#     axs[1,0].annotate(f'+{aep_perc[i+1]:0.2f}%', (p.get_x()+p.get_width()/2, p.get_height()), fontsize=14, ha='center', va=va)



# axs[1,1].set_ylabel('LCOE ($/kWh)')
# axs[1,1].bar(x,lcoe)
# axs[1,1].set_ylim([lcoe.min()*(1-buf),lcoe.max()*(1+buf)])

# for i,p in enumerate(axs[1,1].patches[1:]):
#     if p.get_height() > 0:
#         va = 'bottom'
#     else:
#         va = 'top'
#     axs[1,1].annotate(f'{lcoe_perc[i+1]:0.2f}%', (p.get_x()+p.get_width()/2, p.get_height()), fontsize=14, ha='center', va=va)

    


# [a.set_xticks(x) for a in axs.flatten()]
# # [a.set_xticklabels(['Init.','Level 1','Level 2','Level 3']) for a in axs.flatten()]
# # axs[1,1].set_xticks(x)
# # axs[1,1].set_xticklabels(['Init','Level 1','Level 2','Level 3'])


# fig.set_size_inches(12,8)

# plt.subplots_adjust(wspace=.3)

# fig.align_ylabels()



# plt.bar(x,comp['financese_post.lcoe']/comp['financese_post.lcoe'][0])